In [1]:
import sys

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import pyspark.sql.functions as F

import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.8/dist-packages/pyspark'

In [2]:
import datetime
import pyspark.sql.functions as F
from pyspark.sql.window import Window
 
def input_event_paths(date, depth):
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    return [f"/user/inthemidde/data/events/date={(dt-datetime.timedelta(days=x)).strftime('%Y-%m-%d')}" for x in range(depth)]

def get_spark():
    conf = SparkConf().setAppName(f"EventsPartitioningJob_i028159")
    sc = SparkContext(conf=conf)
    return SQLContext(sc)

def partition_writer(events):
    return events \
        .write \
        .mode('overwrite') \
        .partitionBy('event_type') \
        .format('parquet')

def prepare_parquet(sql, date):
    base_input_path = '/user/master/data/events'
    base_output_path = 'hdfs://rc1a-dataproc-m-dg5lgqqm7jju58f9.mdb.yandexcloud.net/user/inthemidde/data/events'
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    for i in range(0,20,1): #range(start, stop[, step]):       
        lv_date = (dt-datetime.timedelta(days=i)).strftime('%Y-%m-%d')
        print('lv_date=', lv_date)
        events = sql.read.json(f"{base_input_path}/date={lv_date}")
        writer = partition_writer(events)
        writer.save(f'{base_output_path}/date={lv_date}')
        
def run_primary(sql, date):
#     depth = 5
#     dt = datetime.datetime.strptime(date, '%Y-%m-%d')
#     for i in range(0,5,1):
#         print(f'run_primary, iter={i}')
#         rr = tag_tops(dt-datetime.timedelta(days=i), depth, sql)
#         print(f'rr={rr}')
    r = tag_tops('2022-06-04', 5, sql)
    r.repartition(1).write.parquet('/user/inthemidde/data/tmp/tag_tops_06_04_5')


In [6]:
def tag_tops(date, depth, spark):
#     lv_date = date.strftime('%Y-%m-%d')
    print(f' date={date},depth={depth}')
    message_paths = input_event_paths(date, depth)
    print(f' paths={message_paths}')
    result = spark.read\
        .option("basePath", "/user/inthemidde/data/events")\
        .parquet(*message_paths)\
        .where("event_type = 'message'")\
        .where("event.message_channel_to is not null")\
        .select(F.col("event.message_id").alias("message_id"),
                F.col("event.message_from").alias("user_id"),
                F.explode(F.col("event.tags")).alias("tag"))\
        .groupBy("user_id", "tag")\
        .agg(F.count("*").alias("tag_count"))\
        .withColumn("rank", F.row_number().over(Window.partitionBy("user_id")\
        .orderBy(F.desc("tag_count"), F.desc("tag"))))\
        .where("rank <= 3")\
        .groupBy("user_id")\
        .pivot("rank", [1, 2, 3])\
        .agg(F.first("tag"))\
        .withColumnRenamed("1", "tag_top_1")\
        .withColumnRenamed("2", "tag_top_2")\
        .withColumnRenamed("3", "tag_top_3")
    print(f'result={result}')
    return result

In [4]:
sql = get_spark()
print(f'spark={sql}')
prepare_parquet(sql, '2022-06-04')
prepare_parquet(sql, '2022-05-04')

23/08/16 07:25:10 WARN Utils: Your hostname, fhm0o00hgvnb197n9pne resolves to a loopback address: 127.0.1.1; using 172.16.0.4 instead (on interface eth0)
23/08/16 07:25:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/16 07:25:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


spark=<pyspark.sql.context.SQLContext object at 0x7fa720a2c4f0>


/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [10]:
r = tag_tops('2022-06-04', 5, sql)
r.repartition(1).write.parquet('/user/inthemidde/data/tmp/tag_tops_06_04_5')
r = tag_tops('2022-05-04', 5, sql)
r.repartition(1).write.parquet('/user/inthemidde/data/tmp/tag_tops_05_04_5')
r = tag_tops('2022-05-04', 1, sql)
r.repartition(1).write.parquet('/user/inthemidde/data/tmp/tag_tops_05_04_1')


 date=2022-05-04,depth=5
 paths=['/user/inthemidde/data/events/date=2022-05-04', '/user/inthemidde/data/events/date=2022-05-03', '/user/inthemidde/data/events/date=2022-05-02', '/user/inthemidde/data/events/date=2022-05-01', '/user/inthemidde/data/events/date=2022-04-30']
result=DataFrame[user_id: bigint, tag_top_1: string, tag_top_2: string, tag_top_3: string]
 date=2022-05-04,depth=1
 paths=['/user/inthemidde/data/events/date=2022-05-04']
result=DataFrame[user_id: bigint, tag_top_1: string, tag_top_2: string, tag_top_3: string]
